# Set up Custom Dataset for YOLOv4

In [ ]:
DARKNET = r"/mnt/data/cemex/darknet"
DATA = r"/mnt/data/cemex/cx-ir-data/truck_detection/dataset"
CONFIG = r"/mnt/data/cemex/ML\ Training\ scripts/truck_detector/config"

In [ ]:
# copy annotations
yolo_config_name = 'y4t-truckd'
num_classes = 1
width = 416
height = 416
yolo_base_type = 'tiny' #tiny
# yolo_base_type = None # normal

In [ ]:
#Clean darknet folder
!cd {DARKNET} && rm data/obj/*.*

In [ ]:
#Set up training file directories for custom dataset

%cp {CONFIG}/_darknet.labels {DARKNET}/data/obj.names
#%mkdir data/obj
#copy image and labels
%cp {DATA}/*.jpg {DARKNET}/data/obj/
# %cp ~/pmptests/data/val/*.jpg ~/pmptests/darknet/data/obj/

%cp {DATA}/*.txt {DARKNET}/data/obj/
# %cp ~/pmptests/data/val/*.txt ~/pmptests/darknet/data/obj/

## Create data with training info


In [ ]:
%cd {DARKNET}
with open('data/obj.data', 'w') as out:
  out.write('classes = 1\n')
  out.write('train = data/train.txt\n')
  out.write('valid = data/valid.txt\n')
  out.write('names = data/obj.names\n')
  out.write('backup = backup/')

import os
from tqdm.auto import tqdm

split = .9
if False:
    train_files = [f for f in os.listdir(f'{data_folder}/train') if f.endswith('jpg')]
    valid_files = [f for f in os.listdir(f'{data_folder}/val') if f.endswith('jpg')]
else:
    import random
    files  = [ffile for ffile in os.listdir(DATA) if ".txt" not in ffile]
    random.shuffle(files)
    N=len(files)
    train_files = files[:int(N*split)]
    valid_files = files[int(N*split):]
    
    #write train file (just the image list)
    with open('data/train.txt', 'w') as out:
      for img in train_files:
        out.write('data/obj/' + img + '\n')

    #write the valid file (just the image list)
    with open('data/valid.txt', 'w') as out:
      for img in valid_files:
        out.write('data/obj/' + img + '\n')


# Write Custom Training Config for YOLOv4

In [ ]:
pwd

In [ ]:
#Check anchors
#!./darknet detector train data/obj.data cfg/custom-yolov4-tiny-detector.cfg yolov4-tiny.conv.29 -dont_show -map
!./darknet detector calc_anchors data/obj.data -num_of_clusters 6 -width {width} -height {height} -show



In [ ]:
#we build config dynamically based on number of classes
#we build iteratively from base config files. This is the same file shape as cfg/yolo-obj.cfg
%cd {DARKNET}
def file_len(fname):
  with open(fname) as f:
    for i, l in enumerate(f):
      pass
  return i + 1

num_classes = file_len('data/obj.names')
max_batches = num_classes*8000
steps1 = .8 * max_batches
steps2 = .9 * max_batches
steps_str = str(steps1)+','+str(steps2)
num_filters = (num_classes + 5) * 3


print("writing config for a custom YOLOv4 detector detecting number of classes: " + str(num_classes))
#Instructions from the darknet repo
#change line max_batches to (classes*2000 but not less than number of training images, and not less than 6000), f.e. max_batches=6000 if you train for 3 classes
#change line steps to 80% and 90% of max_batches, f.e. steps=4800,5400
if os.path.exists(f'./cfg/{yolo_config_name}.cfg'): os.remove(f'./cfg/{yolo_config_name}.cfg')


#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%writetemplate ./cfg/{yolo_config_name}.cfg
[net]
# Testing
#batch=1
#subdivisions=1
# Training
batch=32
subdivisions=2
width={width}
height={height}
channels=3
momentum=0.9
decay=0.0005
angle=30
flip=0
saturation = 1.5
exposure = 1.5
hue=.1
blur=1
aspect=.75
jitter=0.3
gaussian_noise=1
scale_x_y = 1.05
resize=1.5


learning_rate=0.00261
burn_in=1000
max_batches = {max_batches}
policy=steps
steps={steps_str}
scales=.1,.1

[convolutional]
batch_normalize=1
filters=32
size=3
stride=2
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=64
size=3
stride=2
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=leaky

[route]
layers=-1
groups=2
group_id=1

[convolutional]
batch_normalize=1
filters=32
size=3
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=32
size=3
stride=1
pad=1
activation=leaky

[route]
layers = -1,-2

[convolutional]
batch_normalize=1
filters=64
size=1
stride=1
pad=1
activation=leaky

[route]
layers = -6,-1

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=leaky

[route]
layers=-1
groups=2
group_id=1

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=leaky

[route]
layers = -1,-2

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=leaky

[route]
layers = -6,-1

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=leaky

[route]
layers=-1
groups=2
group_id=1

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=leaky

[route]
layers = -1,-2

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=leaky

[route]
layers = -6,-1

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=512
size=3
stride=1
pad=1
activation=leaky

##################################

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=512
size=3
stride=1
pad=1
activation=leaky

[convolutional]
size=1
stride=1
pad=1
filters={num_filters}
activation=linear



[yolo]
mask = 3,4,5
anchors = 10,14,  23,27,  37,58,  81,82,  135,169,  344,319
#anchors =  13,38,  21,65,  24,102,  34,85,  38,116,  54,123
#anchors =  27, 70,  30, 66,  34, 71,  35, 70,  36, 68,  38, 66
classes={num_classes}
num=6
jitter=.3
scale_x_y = 1.05
cls_normalizer=1.0
iou_normalizer=0.07
iou_loss=ciou
ignore_thresh = .7
truth_thresh = 1
random=0
nms_kind=greedynms
beta_nms=0.6

[route]
layers = -4

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=leaky

[upsample]
stride=2

[route]
layers = -1, 23

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=leaky

[convolutional]
size=1
stride=1
pad=1
filters={num_filters}
activation=linear

[yolo]
mask = 1,2,3
anchors = 10,14,  23,27,  37,58,  81,82,  135,169,  344,319
classes={num_classes}
num=6
jitter=.3
scale_x_y = 1.05
cls_normalizer=1.0
iou_normalizer=0.07
iou_loss=ciou
ignore_thresh = .7
truth_thresh = 1
random=0
nms_kind=greedynms
beta_nms=0.6

In [ ]:
#here is the file that was just written. 
#you may consider adjusting certain things

#like the number of subdivisions 64 runs faster but Colab GPU may not be big enough
#if Colab GPU memory is too small, you will need to adjust subdivisions to 16
%cat cfg/{yolo_config_name}.cfg

# Train Custom YOLOv4 Detector

In [ ]:
!./darknet detector train data/obj.data cfg/{yolo_config_name}.cfg yolov4-tiny.conv.29 -dont_show -map
#If you get CUDA out of memory adjust subdivisions above!
#adjust max batches down for shorter training above

In [ ]:
# from google.colab import files
# files.download('backup/custom-yolov4-tiny-detector_final.weights')

# Infer Custom Objects with Saved YOLOv4 Weights

In [ ]:
#define utility function
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  #plt.rcParams['figure.figsize'] = [10, 5]
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

In [ ]:
#check if weigths have saved yet
#backup houses the last weights for our detector
#(file yolo-obj_last.weights will be saved to the build\darknet\x64\backup\ for each 100 iterations)
#(file yolo-obj_xxxx.weights will be saved to the build\darknet\x64\backup\ for each 1000 iterations)
#After training is complete - get result yolo-obj_final.weights from path build\darknet\x64\bac
%cd darknet
#if it is empty you haven't trained for long enough yet, you need to train for at least 100 iterations

In [ ]:
#coco.names is hardcoded somewhere in the detector
%cp {CONFIG}/obj.names data/coco.names

In [ ]:
#/test has images that we can test our detector on

test_images = [f for f in os.listdir(f'{DATA}') if f.endswith('.jpg')]
import random

for img_path in random.sample(test_images, 10):
    #test out our detector!
    img_path = f"{DATA}/" + img_path;

    !./darknet detect cfg/{yolo_config_name}.cfg backup/{yolo_config_name}_final.weights {img_path} -dont-show
    imShow('predictions.jpg')